In [66]:
import json
import pandas as pd
import numpy as np
from faker import Faker
import random
import psycopg2

### Dataframe authors

In [2]:
#read json and load it to pandas dataframe
df=pd.read_json(r"goodreads_book_authors.json", lines=True) 
df

,average_rating,author_id,text_reviews_count,name,ratings_count
0,3.98,604031,7,Ronald J. Fields,49
1,4.08,626222,28716,Anita Diamant,546796
2,3.92,10333,5075,Barbara Hambly,122118
3,3.68,9212,36262,Jennifer Weiner,888522
4,3.82,149918,96,Nigel Pennick,1740
...,...,...,...,...,...
829524,4.36,197551,4,Patty Furbush,11
829525,4.33,3988103,3,Jim Schlinkman,6
829526,4.00,13464507,2,Rich Jolly,18
829527,3.31,7427847,1,sr@ mwrGn,13


#### author_2010

In [3]:
#check how many unique values we have in 'author_id' to use for primary key
df['author_id'].nunique()

829529

In [4]:
#choose columns that we will use for our author table
author_2010=df[['author_id','name']]

In [5]:
#check for authors without name in frame
len(list(filter(lambda i: i=='',author_2010['name'])))

5

In [6]:
#create nan value where we do not have author names 
#we do not remove them because we need id to suit it with author id from book table
author_2010=author_2010.replace('', 'NULL')

In [7]:
#fake nationality
author_2010['nationality']=[Faker().country() for i in author_2010.index]

In [8]:
#fake gender
#gend=['male','female','other']
#author_2010['gender']=[random.choice(gend) for i in author_2010.index]

In [9]:
author_2010

,author_id,name,nationality
0,604031,Ronald J. Fields,Saint Barthelemy
1,626222,Anita Diamant,American Samoa
2,10333,Barbara Hambly,Mauritania
3,9212,Jennifer Weiner,Saudi Arabia
4,149918,Nigel Pennick,Saint Helena
...,...,...,...
829524,197551,Patty Furbush,Denmark
829525,3988103,Jim Schlinkman,Malaysia
829526,13464507,Rich Jolly,Botswana
829527,7427847,sr@ mwrGn,Lebanon


In [12]:
#Connect to database and insert into author_2010 table values
conn = psycopg2.connect(
   database='comics', user='giorgoslertas', password='1234', host='127.0.0.1', port= '5432'
)
conn.autocommit = True

#Creating cursor object using the cursor() method
cur = conn.cursor()
rows = zip(author_2010.author_id, author_2010.name,author_2010.nationality)
cur.executemany("""INSERT INTO author_2010(id,name,nationality) VALUES (%s, %s, %s)""", rows)

cur.rowcount
cur.close()

# Commit changes in database
conn.commit()
print("Records inserted...")

# Closing the connection
conn.close()

Records inserted...


### Dataframe books

In [13]:
#read json and load it to pandas datafreme
df2=pd.read_json(r"goodreads_books_comics_graphic.json", lines=True)
df2

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,,1,[],US,,"[{'count': '228', 'name': 'to-read'}, {'count'...",B00NLXQ534,true,4.12,,...,,,,https://www.goodreads.com/book/show/25742454-t...,https://s.gr-assets.com/assets/nophoto/book/11...,25742454,1,42749946,The Switchblade Mamma,The Switchblade Mamma
1,2205073346,2,[],US,fre,"[{'count': '2', 'name': 'bd'}, {'count': '2', ...",,false,3.94,,...,1,,2016,https://www.goodreads.com/book/show/30128855-c...,https://images.gr-assets.com/books/1462644346m...,30128855,16,50558228,Cruelle,Cruelle
2,,5,"[246830, 362583, 362581, 623032]",US,eng,"[{'count': '493', 'name': 'to-read'}, {'count'...",,false,4.28,,...,,,2012,https://www.goodreads.com/book/show/13571772-c...,https://images.gr-assets.com/books/1333287305m...,13571772,51,102217,Captain America: Winter Soldier (The Ultimate ...,Captain America: Winter Soldier (The Ultimate ...
3,,1,[],US,eng,"[{'count': '222', 'name': 'to-read'}, {'count'...",B06XKGGSB7,true,4.05,B06XKGGSB7,...,,,,https://www.goodreads.com/book/show/35452242-b...,https://s.gr-assets.com/assets/nophoto/book/11...,35452242,6,54276229,Bounty Hunter 4/3: My Life in Combat from Mari...,Bounty Hunter 4/3: My Life in Combat from Mari...
4,0930289765,6,"[266759, 1096220]",US,en-US,"[{'count': '20', 'name': 'to-read'}, {'count':...",,false,4.06,,...,11,,1997,https://www.goodreads.com/book/show/707611.Sup...,https://images.gr-assets.com/books/1307838888m...,707611,51,693886,"Superman Archives, Vol. 2","Superman Archives, Vol. 2"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89406,,22,[],US,,"[{'count': '114', 'name': 'to-read'}, {'count'...",B01GENHTK0,true,3.97,B01GENHTK0,...,,,,https://www.goodreads.com/book/show/30848889-d...,https://s.gr-assets.com/assets/nophoto/book/11...,30848889,338,51454503,Doctor Who: Free Comic Book Day 2016,Doctor Who: Free Comic Book Day 2016
89407,,2,[669800],US,,"[{'count': '620', 'name': 'to-read'}, {'count'...",B01HDRTT5Y,true,3.86,B01HDRTT5Y,...,,,,https://www.goodreads.com/book/show/33553114-love,https://s.gr-assets.com/assets/nophoto/book/11...,33553114,4,40074057,Love: The Tiger,Love: The Tiger
89408,,5,[638272],US,spa,"[{'count': '52', 'name': 'to-read'}, {'count':...",,false,3.87,,...,11,Integral Edition,2013,https://www.goodreads.com/book/show/20653416-l...,https://images.gr-assets.com/books/1391037254m...,20653416,28,19161997,La búsqueda del pájaro del tiempo,La búsqueda del pájaro del tiempo
89409,,2,[],US,eng,"[{'count': '1', 'name': '3-2017-ha-pages-read-...",,true,3.00,,...,2,,2016,https://www.goodreads.com/book/show/28928706-d...,https://images.gr-assets.com/books/1454539757m...,28928706,10,49145631,Dejah Thoris #1,Dejah Thoris #1


#### publisher_2010

In [14]:
#check how many unique values we have in 'publisher'
df2.publisher.nunique()

7129

In [15]:
#remove values without publisher name, we want unique values
publisher_2010=pd.DataFrame(df2.publisher.unique(), columns=['publisher'])
publisher_2010

,publisher
0,
1,Dargaud
2,Hachette Partworks Ltd.
3,DC Comics
4,Go! Comi
...,...
7124,Serieframjandet
7125,Fernandel
7126,Besna Kobila
7127,FL!P Comics


In [16]:
#create fake values for publisher and add them
publisher_2010['address']=[Faker().address() for i in range(len(publisher_2010))]
publisher_2010['country']=[Faker().country() for i in range(len(publisher_2010))]
publisher_2010['pub_phone']=[Faker().phone_number() for i in range(len(publisher_2010))]

In [17]:
#Connect to database and insert into publisher table values
conn = psycopg2.connect(
   database='comics', user='giorgoslertas', password='1234', host='127.0.0.1', port= '5432'
)
conn.autocommit = True

#Creating cursor object using the cursor() method
cur = conn.cursor()
rows = zip(publisher_2010.publisher,publisher_2010.address,publisher_2010.country,publisher_2010.pub_phone)
cur.executemany("""INSERT INTO publisher_2010(name, address, country, pub_phone) VALUES (%s, %s, %s, %s)""", rows)

cur.rowcount
cur.close()

# Commit changes in database
conn.commit()
print("Records inserted...")

# Closing the connection
conn.close()

Records inserted...


#### book_2010 

In [18]:
#check how many unique values we have in 'book_id' to use for primary key
df2['book_id'].nunique()

89411

In [19]:
#choose columns that we will use for our book table, where we have isbn
book_2010 = df2[['book_id','isbn','title','publisher','publication_year','description']].loc[df2['isbn'] != '']

In [20]:
#dataframe with length of titles
dm=pd.DataFrame([len(i) for i in book_2010['title']])

In [21]:
#2 titles over our length (200), maybe we must create varchar(300)
dm[dm[0]>199]

,0
10835,237
16105,253


In [22]:
#search in attributes for missing values
len(list(filter(lambda i: i=='',book_2010['description'])))

7326

In [23]:
#create a message for description where we have missing values
book_2010['description']=book_2010['description'].replace('', 'There is no description' )

In [25]:
#create random prices
book_2010['price']=[random.choice(np.arange(5,100,0.5)) for i in book_2010.index]

In [28]:
book_2010

,book_id,isbn,title,publisher,publication_year,description,price
1,30128855,2205073346,Cruelle,Dargaud,2016,"Florence Dupre Latour raconte comment, de son ...",50.0
4,707611,0930289765,"Superman Archives, Vol. 2",DC Comics,1997,These are the stories that catapulted Superman...,69.5
5,2250580,1933617640,"A.I. Revolution, Vol. 1",Go! Comi,2007,"Like everyone else in the future, Sui's used t...",89.0
6,27036536,1592912729,"War Stories, Volume 3",Avatar Press,2016,PRODUCT DESCRIPTION: The first new volume of G...,46.5
7,27036537,1592912737,"Crossed, Volume 15",Avatar Press,2016,Comics horror veteran Mike Wolfer writes and i...,31.0
...,...,...,...,...,...,...,...
89398,7657484,1401224687,"Superman: Earth One, Volume 1",DC Comics,2010,Forget everything you know about The Man of St...,27.0
89400,30112514,1503938999,Mayhem and Magic,Jet City Comics,2016,Freelance thief-for-hire Asa Ward's latest job...,27.5
89401,6419558,1421526123,"Kekkaishi, Vol. 19 (Kekkaishi, #19)",VIZ Media LLC,2009,"Student by day, demon hunter by night! R to L ...",90.5
89403,3106983,009952399X,Persepolis: The Story of a Childhood and The S...,Vintage,2008,The Story of a Childhood and The Story of a Re...,20.0


In [30]:
#Connect to database and insert into book_2010 table values
conn = psycopg2.connect(
   database='comics', user='giorgoslertas', password='1234', host='127.0.0.1', port= '5432'
)
conn.autocommit = True

#Creating cursor object using the cursor() method
cur = conn.cursor()
rows = zip(book_2010.book_id,book_2010.isbn,book_2010.title,book_2010.publisher,book_2010.publication_year,book_2010.description,book_2010.price)
cur.executemany("""INSERT INTO book_2010(id,isbn,title,publisher,publ_year,description,price) VALUES (%s, %s, %s, %s, %s, %s, %s)""", rows)
cur.rowcount
cur.close()

# Commit changes in database
conn.commit()
print("Records inserted...")
# Closing the connection
conn.close()

Records inserted...


#### author_book_2010

In [31]:
#remove isbn without value
df2=df2.loc[df2['isbn'] != '']

In [32]:
#create dataframe for books and authors
author_book_2010=pd.DataFrame(columns=['book_id','author_id','role'])
book=[]
auth=[]
role=[]

#create dataframe with three lists to take values from df2
for index, row in df2.iterrows():
    for i in row.authors:
        book.append(row['book_id'])
        auth.append(list(i.values())[0])
        role.append(list(i.values())[1])

author_book_2010['book_id']=book
author_book_2010['author_id']=auth
author_book_2010['role']=role

In [33]:
#create nan values where we have missing values
author_book_2010=author_book_2010.replace('', 'NULL')

In [34]:
#delete duplicate rows from table
author_book_2010=author_book_2010.drop_duplicates()

In [35]:
author_book_2010

,book_id,author_id,role
0,30128855,3274315,NULL
1,707611,81563,NULL
2,707611,89537,Illustrator
3,2250580,1015982,NULL
4,27036536,14965,NULL
...,...,...,...
129678,3106983,6238,NULL
129679,3106983,3141986,Translator
129680,10644600,13557729,Adaptor
129681,10644600,346732,Story


In [36]:
#Connect to database and insert into author_book_2010 table values
conn = psycopg2.connect(
   database='comics', user='giorgoslertas', password='1234', host='127.0.0.1', port= '5432'
)
conn.autocommit = True

#Creating cursor object using the cursor() method
cur = conn.cursor()
rows = zip(author_book_2010.book_id,author_book_2010.author_id,author_book_2010.role)
cur.executemany("""INSERT INTO author_book_2010(book_id,author_id,role) VALUES (%s, %s, %s)""", rows)

cur.rowcount
cur.close()

# Commit changes in database
conn.commit()
print("Records inserted...")

# Closing the connection
conn.close()

Records inserted...


### Dataframe reviews 

In [37]:
#read json and load it to pandas datafreme
df3=pd.read_json(r"goodreads_reviews_comics_graphic.json", lines=True)
df3

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,dc3763cdb9b2cae805882878eebb6a32,18471619,66b2ba840f9bd36d6d27f46136fe4772,3,Sherlock Holmes and the Vampires of London \n ...,Thu Dec 05 10:44:25 -0800 2013,Thu Dec 05 10:45:15 -0800 2013,Tue Nov 05 00:00:00 -0800 2013,,0,0
1,bafc2d50014200cda7cb2b6acd60cd73,6315584,72f1229aba5a88f9e72f0dcdc007dd22,4,"I've never really liked Spider-Man. I am, howe...",Wed Aug 10 06:06:48 -0700 2016,Fri Aug 12 08:49:54 -0700 2016,Fri Aug 12 08:49:54 -0700 2016,Wed Aug 10 00:00:00 -0700 2016,0,0
2,bafc2d50014200cda7cb2b6acd60cd73,29847729,a75309355f8662caaa5e2c92ab693d3f,4,"A very quick introduction, this is coming out ...",Thu Apr 21 07:44:00 -0700 2016,Thu Apr 21 07:59:28 -0700 2016,Thu Apr 21 07:59:28 -0700 2016,Thu Apr 21 00:00:00 -0700 2016,0,0
3,bafc2d50014200cda7cb2b6acd60cd73,18454118,c3cc5a3e1d6b6c9cf1c044f306c8e752,5,I've been waiting so long for this. I first st...,Mon Mar 03 17:45:56 -0800 2014,Mon Mar 03 17:54:11 -0800 2014,Sat Mar 01 00:00:00 -0800 2014,Sat Mar 01 00:00:00 -0800 2014,1,0
4,bafc2d50014200cda7cb2b6acd60cd73,2239435,cc444be37ab0a42bfb4dd818cb5edd10,4,The only thing more entertaining than this boo...,Wed Apr 03 12:37:48 -0700 2013,Wed Apr 03 13:03:36 -0700 2013,Wed Apr 03 13:03:36 -0700 2013,,0,0
...,...,...,...,...,...,...,...,...,...,...,...
542333,c9174473efac302c71adaa1ad2a46091,20176586,8e069a687dbecae9ffe7f5fa8772ac08,3,Bumping this up to a three star rating primari...,Tue Jan 26 20:42:40 -0800 2016,Tue Jan 26 20:46:31 -0800 2016,Wed Jan 27 20:46:31 -0800 2016,Tue Jan 26 00:00:00 -0800 2016,0,0
542334,64946797934367a178abe75e3febce66,11465282,77c383db515b63f84ad0f5de0cb5991d,3,The story didn't pull me in although I'm aware...,Wed Mar 18 06:23:25 -0700 2015,Sun Mar 29 12:54:35 -0700 2015,Sun Mar 29 12:54:35 -0700 2015,Wed Mar 18 00:00:00 -0700 2015,0,0
542335,bd575ed739592a8c20c346e5daa5ac76,31094296,9e059a74a7107fb57f5b45ded1abadb6,5,Stunning follow up. Once again it's not the ar...,Mon Oct 02 07:42:53 -0700 2017,Mon Oct 02 07:44:18 -0700 2017,,,0,0
542336,bd575ed739592a8c20c346e5daa5ac76,27404461,de2a13cd890ead821d846398239d5f1f,5,I grew up with the Terran Trade Authority book...,Sun Dec 06 12:34:17 -0800 2015,Mon Oct 02 07:42:28 -0700 2017,Sun Dec 06 12:39:44 -0800 2015,Wed Dec 02 00:00:00 -0800 2015,0,0


#### review_2010

In [38]:
#check how many unique values we have in 'review_id' to use for primary key
df3['review_id'].nunique()

542338

In [39]:
#choose columns that we will use for our review table
review_2010=df3[['review_id','book_id','rating','review_text','date_added']]

In [40]:
#run it for all attributes and only 'review_text' has missing values
len(list(filter(lambda i: i=='',review_2010['review_text'])))

323

In [41]:
#create nan values where we have missing values
review_2010=review_2010.replace('', 'NULL')

In [42]:
#merge reviews with book_2010.book_id to remove reviews without isbn
review_2010=pd.merge(book_2010.book_id,review_2010,how='inner',on='book_id')

In [43]:
#create nicknames
review_2010['nickname']=[Faker().last_name() for i in review_2010.index]

In [44]:
review_2010

,book_id,review_id,rating,review_text,date_added,nickname
0,30128855,6133733899c4a1aa8885b3be42e03c22,4,"Un livre troublant, une oeuvre a decouvrir.",Sun Aug 14 09:49:50 -0700 2016,Anderson
1,707611,c625d2be24e875780ad99f4ac25e6e3a,3,action 80,Thu Jan 07 00:32:16 -0800 2016,Parker
2,2250580,e2e95a5a915fb9dde40588f8749af632,1,"Midori's father makes robots, and his latest i...",Mon Feb 02 15:21:26 -0800 2009,Cabrera
3,2250580,6b4c4f7c6d75d63bb1b1211d5d59b2c7,3,It is about a girl and her father makes robots...,Mon Nov 03 10:49:16 -0800 2008,Little
4,2250580,9ae1a9e53833a35abc2e7c6e28fc8a14,3,"Meh, series is kind of boring, I dropped it.",Fri Aug 25 12:00:43 -0700 2017,Wilson
...,...,...,...,...,...,...
461923,10644600,a24bd4e91da9704257a11230a4900537,4,"Is it me, or does George R.R. Martin have some...",Mon Oct 17 12:32:14 -0700 2011,Murillo
461924,10644600,9078bbfcd22e2922374d2616f339efdf,4,3.5,Wed Feb 11 12:33:33 -0800 2015,Davis
461925,10644600,7c5672d72658eb561d4772514aa766a5,4,I enjoyed this book. It a different writing st...,Tue Jul 08 18:56:08 -0700 2014,Weiss
461926,10644600,2eea60d4a18ee36b9d3a6efb3d7296e7,3,"may have to pick this one up. and when i say ""...",Sun Sep 04 09:55:00 -0700 2011,Acosta


In [45]:
#Connect to database and insert into review_2010 table values
conn = psycopg2.connect(
   database='comics', user='giorgoslertas', password='1234', host='127.0.0.1', port= '5432'
)
conn.autocommit = True

#Creating cursor object using the cursor() method
cur = conn.cursor()
rows = zip(review_2010.review_id,review_2010.nickname,review_2010.book_id,review_2010.rating,review_2010.review_text,review_2010.date_added)
cur.executemany("""INSERT INTO review_2010(id,nickname,book_id,rating,comment,date_added) VALUES (%s, %s, %s, %s, %s, %s)""", rows)

cur.rowcount
cur.close()

# Commit changes in database
conn.commit()
print("Records inserted...")

# Closing the connection
conn.close()

Records inserted...


#### user_2010

In [48]:
#create dataframe user_2010
user_2010=pd.DataFrame()
#create fake values for user and add them
n=10
user_2010['id']=[random.choice(df3['user_id']) for i in range(n)]
user_2010['username']=[Faker().first_name() for i in range(n)]
user_2010['password']=[Faker().password() for i in range(n)]
user_2010['email']=[Faker().email() for i in range(n)]
user_2010['name']=[Faker().name() for i in range(n)]
user_2010['user_phone']=[Faker().phone_number() for i in range(n)]
user_2010

,id,username,password,email,name,user_phone
0,18cd09a0c7ddf5cbf2fc6eb15c2836fe,Alexander,l4EJX)Py&1,tiffany28@gmail.com,Roger Walker,909.720.7158x735
1,2665c8f0bbd66b70caf2c3fc78ee96cf,Stephanie,*3PEuWkxxD,tcarson@hotmail.com,Jessica Garcia,355.760.2467x1526
2,c67aa06c32d5ca4193d8e11c351dc7d8,Jason,N%3vSVzz2L,morrischase@martin.com,Patricia Vang,(890)886-7413x9642
3,dc36c784155965c8e4986a13719a35fd,Courtney,%4$#UTNblG,bakermatthew@baker-patterson.com,Danielle Salazar,313.863.3305
4,9453d59ff8e823e93833af73969177c7,Phillip,$h&1CeOC_a,ghiggins@yahoo.com,Melinda Jones,001-127-767-3165x6323
5,5a9a24ba83f08516c57c198fe52c6f96,Shannon,n@_7i9Mnx1,emily91@ruiz-orozco.com,Michael Torres,3072499040
6,e051fd48f6fdb4a5ffd0522526529c39,Pamela,2SYNRQdt^@,cameronlopez@phillips-wright.org,Laurie Walker,549-207-8966x94214
7,dd0a518c3d7bf43a505da2a81da95365,Tammy,_fmmMT%964,lori98@hunt.com,Maria White,(172)014-5493
8,2b2009bde189e857d93f7fe7384f6ee8,Emily,8*G3Iaxd^D,scotttaylor@gmail.com,Joshua Fisher,+1-623-667-0868
9,b48ef05b19ab6367ac614de4b3ecd6f8,Cody,%$DF$(HvX3,zking@lang.biz,Frederick Stanley,(050)084-3918


In [49]:
#Connect to database and insert into user table values
conn = psycopg2.connect(
   database='comics', user='giorgoslertas', password='1234', host='127.0.0.1', port= '5432'
)
conn.autocommit = True

#Creating cursor object using the cursor() method
cur = conn.cursor()
rows = zip(user_2010.id,user_2010.username,user_2010.password,user_2010.email,user_2010.name,user_2010.user_phone)
cur.executemany("""INSERT INTO user_2010(id,username,password,email,name,user_phone) VALUES (%s, %s, %s, %s, %s, %s)""", rows)

cur.rowcount
cur.close()

# Commit changes in database
conn.commit()
print("Records inserted...")

# Closing the connection
conn.close()

Records inserted...


#### addresses_2010

In [50]:
#create dataframe for user addresses
addresses_2010=pd.DataFrame()
addresses_2010['user_id']=user_2010.id
addresses_2010['address']=[Faker().address() for i in range(n)]
addresses_2010

,user_id,address
0,18cd09a0c7ddf5cbf2fc6eb15c2836fe,"267 Jeremy Lake Apt. 297\nTaylorfort, WY 13966"
1,2665c8f0bbd66b70caf2c3fc78ee96cf,"6046 Lisa Row\nParkerfurt, CA 63568"
2,c67aa06c32d5ca4193d8e11c351dc7d8,"398 Robert Highway\nMillerbury, RI 51569"
3,dc36c784155965c8e4986a13719a35fd,Unit 4514 Box 9686\nDPO AE 24073
4,9453d59ff8e823e93833af73969177c7,"390 Figueroa Extension Suite 138\nWest Karen, ..."
5,5a9a24ba83f08516c57c198fe52c6f96,"172 King Club\nEast Matthewview, CA 07059"
6,e051fd48f6fdb4a5ffd0522526529c39,"377 Brianna Trafficway\nCastillomouth, DE 90945"
7,dd0a518c3d7bf43a505da2a81da95365,"768 Gregory Expressway Suite 910\nKimside, CT ..."
8,2b2009bde189e857d93f7fe7384f6ee8,USS Williams\nFPO AE 77251
9,b48ef05b19ab6367ac614de4b3ecd6f8,USCGC Sanders\nFPO AE 24885


In [51]:
#Connect to database and insert into addresses_2010 table values
conn = psycopg2.connect(
   database='comics', user='giorgoslertas', password='1234', host='127.0.0.1', port= '5432'
)
conn.autocommit = True

#Creating cursor object using the cursor() method
cur = conn.cursor()
rows = zip(addresses_2010.user_id,addresses_2010.address)
cur.executemany("""INSERT INTO addresses_2010(user_id,address) VALUES (%s, %s)""", rows)

cur.rowcount
cur.close()

# Commit changes in database
conn.commit()
print("Records inserted...")

# Closing the connection
conn.close()

Records inserted...


#### orders_2010

In [69]:
#create dataframe as table orders_2010
orders_2010=pd.DataFrame()
orders_2010[['user_id','user_addr']]= addresses_2010.sample(n=100,axis=0,replace = True) #100 orders
orders_2010                                                     

,user_id,user_addr
5,5a9a24ba83f08516c57c198fe52c6f96,"172 King Club\nEast Matthewview, CA 07059"
9,b48ef05b19ab6367ac614de4b3ecd6f8,USCGC Sanders\nFPO AE 24885
7,dd0a518c3d7bf43a505da2a81da95365,"768 Gregory Expressway Suite 910\nKimside, CT ..."
9,b48ef05b19ab6367ac614de4b3ecd6f8,USCGC Sanders\nFPO AE 24885
4,9453d59ff8e823e93833af73969177c7,"390 Figueroa Extension Suite 138\nWest Karen, ..."
...,...,...
1,2665c8f0bbd66b70caf2c3fc78ee96cf,"6046 Lisa Row\nParkerfurt, CA 63568"
8,2b2009bde189e857d93f7fe7384f6ee8,USS Williams\nFPO AE 77251
2,c67aa06c32d5ca4193d8e11c351dc7d8,"398 Robert Highway\nMillerbury, RI 51569"
5,5a9a24ba83f08516c57c198fe52c6f96,"172 King Club\nEast Matthewview, CA 07059"


In [74]:
#Connect to database and insert into orders_2010 table values
conn = psycopg2.connect(
   database='comics', user='giorgoslertas', password='1234', host='127.0.0.1', port= '5432'
)
conn.autocommit = True

#Creating cursor object using the cursor() method
cur = conn.cursor()
rows = zip(orders_2010.user_id,orders_2010.user_addr)
cur.executemany("""INSERT INTO orders_2010(user_id,user_addr) VALUES (%s, %s)""", rows)

cur.rowcount
cur.close()

# Commit changes in database
conn.commit()
print("Records inserted...")

# Closing the connection
conn.close()

Records inserted...


#### order_books_2010

In [71]:
#create dataframe as table order_books_2010
order_books_2010=pd.DataFrame()
order_books_2010['book_id']=list(book_2010['book_id'].sample(n=150,axis=0)) #150 books for 100 orders
order_books_2010['order_id']=[random.choice(range(100))+1 for i in range(150)]
order_books_2010

,book_id,order_id
0,6936599,81
1,15798397,75
2,448698,19
3,3083278,85
4,28375750,88
...,...,...
145,1767728,56
146,22529472,71
147,7717390,39
148,8606109,51


In [75]:
#Connect to database and insert into order_books_2010 table values
conn = psycopg2.connect(
   database='comics', user='giorgoslertas', password='1234', host='127.0.0.1', port= '5432'
)
conn.autocommit = True

#Creating cursor object using the cursor() method
cur = conn.cursor()
rows = zip(order_books_2010.book_id,order_books_2010.order_id)
cur.executemany("""INSERT INTO order_books_2010(book_id,order_id) VALUES (%s, %s)""", rows)

cur.rowcount
cur.close()

# Commit changes in database
conn.commit()
print("Records inserted...")

# Closing the connection
conn.close()

Records inserted...
